<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# `pandas` Data Munging Overview: Part 2

_Authors: Joseph Nelson (DC)_

---

**Warning: This is a resource-heavy notebook that can consume a lot of RAM, especially when it's run in Chrome. For this lesson, you may want to close idle applications and/or open this notebook with Safari.**

### Lesson Guide
- [Exercise #3](#exercise-3)
- [Split-Apply-Combine](#split-apply-combine)
    - [`.groupby()`](#groupby)
    - [Apply Functions to Groups and Combine](#apply-combine)
- [Exercise #4](#exercise-4)
- [Indexing](#indexing)
    - [Location Indexing With `.loc()`](#loc)
    - [Position Indexing With `.iloc()`](#iloc)
- [Other Frequently Used Features](#frequent)
    - [Using Map Functions With Replacement Dictionaries](#map-dict)
    - [Encoding Strings as Integers With `.factorize()`](#factorize)
    - [Determining Unique Values](#unique)
    - [Replacing Values With `.replace()`](#replace)
    - [Series String Methods With `.str`](#series-str)
    - [Datetime Conversion and Arithmetic](#datetime)
    - [Setting and Resetting the Index](#set-reset-index)
    - [Sorting by Index](#sort-by-index)
    - [Changing the Data Type of a Column](#change-dtype)
    - [Creating Dummy-Coded Columns](#dummy)
    - [Concatenating DataFrames](#concatenate)
    - [Detecting and Dropping Duplicate Rows](#duplicate-rows)
    - [Writing a DataFrame to a `.csv`](#write-csv)
    - [Pickling a DataFrame](#pickle)
    - [Randomly Sampling a DataFrame](#sample)
- [Infrequently Used Features](#infrequent)
    - [Creating DataFrames From Dictionaries and Lists of Lists](#toy-dataframes)
    - [Performing Cross-Tabulations](#crosstab)
    - [Query-Filtering Syntax](#query)
    - [Calculating Memory Usage](#memory-usage)
    - [Converting Column to Category Type](#category-type)
    - [Creating Columns With `.assign()`](#assign)
    - [Limiting the Number of Rows to Load in a File Read](#limit-rows-read)
    - [Manually Setting the Number of Rows and Columns to Print](#manual-print)

In [ ]:
import pandas as pd

<a id='exercise-3'></a>
## Exercise #3

---

**Using the UFO data provided below:**
1. Read in the data.
2. Check the shape and describe the columns.
3. Find the four most frequently reported colors.
4. Find the most frequent city for reports in state `VA`.
5. Find only UFO reports from Arlington, VA.
6. Find the number of missing values in each column.
7. Show only UFO reports where `city` is missing.
8. Count the number of rows with no null values.
9. Amend column names with spaces to have underscores.
10. Make a new column that is a combination of `city` and `state`.

In [ ]:
ufo_csv = 'https://git.generalassemb.ly/dsi-unit-2/pandas-data_munging_full_overview-lesson/tree/master/datasets/ufo.csv'
ufo_csv = '../datasets/ufo.csv'

In [ ]:
# Read `ufo.csv` into a DataFrame called `ufo`.
ufo = pd.read_table(ufo_csv, sep=',')
ufo = pd.read_csv(ufo_csv)

In [ ]:
# Check the shape of the DataFrame.
ufo.shape

In [ ]:
# Calculate the most frequent value for each of the columns in a single command.
ufo.describe()

In [ ]:
# What are the four most frequently reported colors?
ufo['Colors Reported'].value_counts().head(4)

In [ ]:
# For reports in `VA`, what's the most frequently listed city?
ufo[ufo.State=='VA'].City.value_counts().head(1)

In [ ]:
# Show only the UFO reports from Arlington, VA.
ufo[(ufo.City=='Arlington') & (ufo.State=='VA')]

In [ ]:
# Count the number of missing values in each column.
ufo.isnull().sum()

In [ ]:
# Show only the UFO reports in which the `city` is missing.
ufo[ufo.City.isnull()]

In [ ]:
# How many rows remain if you drop all rows with any missing values?
ufo.dropna().shape[0]

In [ ]:
# Replace any spaces in the column names with underscores.
ufo.rename(columns={'Colors Reported':'Colors_Reported', 'Shape Reported':'Shape_Reported'}, inplace=True)


In [ ]:
# BONUS: Redo the task above, writing generic code to replace spaces with underscores.
# In other words, your code should not reference the specific column names.
ufo.columns = [col.replace(' ', '_') for col in ufo.columns]
ufo.columns = ufo.columns.str.replace(' ', '_')

In [ ]:
# Create a new column called `location` that includes both `city` and `state`.
# For example, the `location` for the first row would be `Ithaca, NY`.
ufo['Location'] = ufo.City + ', ' + ufo.State

<a id='split-apply-combine'></a>
## Split-Apply-Combine

---

![](../assets/split_apply_combine.png)

<a id='groupby'></a>
### `.groupby()`

**Q.1** Using the `drinks` DataFrame, calculate the mean `beer` servings by continent.

In [ ]:
drinks =pd.read_csv('../datasets/drinks_updated.csv')

In [ ]:
# For each continent, calculate the mean `beer` servings.
drinks.groupby('continent').beer.mean()

**Q.2** Describe the `beer` column by continent.

In [ ]:
# For each continent, describe `beer` servings.
drinks.groupby('continent').beer.describe()

<a id='apply-combine'></a>
### Apply Functions to Groups and Combine

**Q.1** Find the `count`, `mean`, `minimum`, and `maximum `of the `beer` column by continent.

In [ ]:
# Similar, this but outputs a DataFrame and can be customized.
drinks.groupby('continent').beer.agg(['count', 'mean', 'min', 'max'])

**Q.2** Perform the same task as in Q.1, but now sort the output by the `mean` column.

In [ ]:
drinks.groupby('continent').beer.agg(['count', 'mean', 'min', 'max']).sort_values('mean')


**Q.3** Apply a custom function to all columns of the `drinks` DataFrame, grouping by continent.

In [ ]:
# Find the first value of each column by continent:
drinks.groupby('continent').apply(lambda x: x.iloc[0,:])

**Q.4** **Note:** If you don't specify a column for the aggregation function, it will be applied to all numeric columns.

In [ ]:
drinks.groupby('continent').mean()
drinks.groupby('continent').describe()

<a id='exercise-4'></a>

## Exercise #4

---

**Using the `users` DataFrame**:
1. Count the number of distinct occupations in `users`.
2. Calculate the mean age by occupation.
3. Calculate the minimum and maximum age by occupation.
4. Calculate the mean age by cross-sections of `occupation` and `gender`.

> **Tip**: Multiple columns can be passed to the `.groupby()` function for more granular cross-sections.

In [ ]:
users=pd.read_table('../datasets/users.txt', sep='|')

In [ ]:
# For each occupation in `users`, count the number of occurrences.
users.occupation.value_counts()

In [ ]:
# For each occupation, calculate the mean age.
users.groupby('occupation').age.mean()

In [ ]:
# For each occupation, calculate the minimum and maximum ages.
users.groupby('occupation').age.agg(['min', 'max'])

In [ ]:
# For each combination of `occupation` and `gender`, calculate the mean age.
users.groupby(['occupation', 'gender']).age.mean()

<a id='indexing'></a>
## Indexing

---
<a id='loc'></a>
### Location Indexing With `.loc()`

**Q.1** Select all rows and the `city` column from the UFO data set using `.loc()`.

In [ ]:
d = ufo.loc[:, 'City'] # Colon means "all rows;" then, select one column
d

**Q.2** Select all rows and columns in `city` and `state`.

In [ ]:
d = ufo.loc[:, ['City', 'State']]   # Select two columns
d

**Q.3** Select all rows and columns from `city` *through* `state`.

In [ ]:
d = ufo.loc[:, 'City':'State'] # Select a range of columns.
d.columns

**Q.4** Select:
- All columns at row 0.
- All columns at rows 0:2.
- Columns `city` through `state` at rows 0:2.

In [ ]:
# `.loc()` can also filter rows by "name" (the index).
d = ufo.loc[0, :]                   # Row 0, all columns
d = ufo.loc[0:2, :]                 # Rows 0/1/2, all columns
d = ufo.loc[0:2, 'City':'State']    # Rows 0/1/2, range of columns

<a id='iloc'></a>
### Position indexing with `.iloc`

**Q.1** Select all rows and columns in position 0 and 3.

In [ ]:
d = ufo.iloc[:, [0, 3]] # All rows, columns in position 0/3
d

**Q.2** Select all rows and columns in positions 0 through 4.

In [ ]:
d = ufo.iloc[:, 0:4] # All rows, columns in position 0/1/2/3
d

**Q.3** Select rows in positions 0:3, along with all columns.

In [ ]:
d = ufo.iloc[0:3, :] # rows in position 0/1/2, all columns
d

<a id='frequent'></a>
## Frequently Used Features

---
<a id='map-dict'></a>
### Using Map Functions With Replacement Dictionaries

In [ ]:
# Map existing values to a different set of values.
users['is_male'] = users.gender.map({'F':0, 'M':1})

<a id='factorize'></a>
### Encoding Strings as Integers With `.factorize()`

In [ ]:
# Encode strings as integer values. (This function automatically starts at 0).
users['occupation_num'] = users.occupation.factorize()[0]

users.head()

<a id='unique'></a>
### Determining Unique Values

In [ ]:
# Determine unique values in a column.
users.occupation.nunique()      # Count the number of unique values.

In [ ]:
users.occupation.unique()       # Return the unique values.

<a id='replace'></a>
### Replacing Values With `.replace()`

In [ ]:
# Replace all instances of a value in a column (must match the entire value).
ufo.State.replace('Fl', 'FL', inplace=True)

<a id='series-str'></a>
### Series String Methods With `.str`

In [ ]:
# String methods are accessed via `.str`.
ufo.State.str.upper()                               # Converts to uppercase
ufo.Colors_Reported.str.contains('RED', na='False').head(2) # Checks for a substring

<a id='datetime'></a>
### Datetime Conversion and Arithmetic

In [ ]:
# Convert a string to the datetime format.
ufo['Time'] = pd.to_datetime(ufo.Time)
ufo.Time.dt.hour                        # Datetime format exposes convenient attributes.
(ufo.Time.max() - ufo.Time.min()).days  # It also allows you to do datetime "math."
ufo[ufo.Time > pd.datetime(2014, 1, 1)].head(2) # Boolean filtering with the datetime format

<a id='set-reset-index'></a>
### Setting and Resetting the Index

In [ ]:
# Setting and then removing an index
ufo.set_index('Time', inplace=True)
ufo.reset_index(inplace=True)

<a id='sort-by-index'></a>
### Sorting by Index

In [ ]:
# Sort a column by its index.
ufo.State.value_counts().sort_index()[0:3]

<a id='change-dtype'></a>
### Changing the Data Type of a Column

In [ ]:
# Change the data type of a column.
drinks['beer'] = drinks.beer.astype('float')

# Change the data type of a column when reading in a file.
d = pd.read_csv('../datasets/drinks.csv', dtype={'beer_servings':float})

<a id='dummy'></a>
### Creating Dummy-Coded Columns

In [ ]:
# Create dummy variables for `continent` and exclude the first dummy column.
continent_dummies = pd.get_dummies(drinks.continent, prefix='cont').iloc[:, 1:]
continent_dummies.head(3)

<a id='concatenate'></a>
### Concatenating DataFrames

In [ ]:
# Concatenate two DataFrames (axis=0 for rows, axis=1 for columns).
drinks = pd.concat([drinks, continent_dummies], axis=1)

In [ ]:
drinks.head(2)

<a id='duplicate-rows'></a>
### Detecting and Dropping Duplicate Rows

In [ ]:
# Detecting duplicate rows:
d = users.duplicated()          # True if a row is identical to a previous row.
d = users.duplicated().sum()    # Count of duplicates.
d = users[users.duplicated()]   # Only shows duplicates.
d = users.drop_duplicates()     # Drops duplicate rows.
d = users.age.duplicated()      # Checks a single column for duplicates.
d = users.duplicated(['age', 'gender', 'zip_code']).sum()   # Specifies columns for finding duplicates.

<a id='write-csv'></a>
### Writing a DataFrame to a `.csv`
```python
# Write a DataFrame out to a `.csv`.
drinks.to_csv('drinks_updated.csv')  # Index is used as the first column
drinks.to_csv('drinks_updated.csv', index=False) # Ignore index
```

<a id='pickle'></a>
### Pickling a DataFrame
```python
# Save a DataFrame to disk (a.k.a., "pickle") and read it from disk (a.k.a., "unpickle").
drinks.to_pickle('drinks_pickle')
pd.read_pickle('drinks_pickle')
```

<a id='sample'></a>
### Randomly Sampling a DataFrame

In [ ]:
# Randomly sample a DataFrame.
train = drinks.sample(frac=0.75, random_state=1)    # Will contain 75% of the rows
test = drinks[~drinks.index.isin(train.index)]      # Will contain the other 25%

<a id='infrequent'></a>
## Infrequently Used Features

---

<a id='toy-dataframes'></a>
### Creating DataFrames From Dictionaries and Lists of Lists

In [ ]:
# Create a DataFrame from a dictionary.
d = pd.DataFrame({'capital':['Montgomery', 'Juneau', 'Phoenix'], 'state':['AL', 'AK', 'AZ']})
d.head(2)

In [ ]:
# Create a DataFrame from a list of lists.
d = pd.DataFrame([['Montgomery', 'AL'], ['Juneau', 'AK'], ['Phoenix', 'AZ']], columns=['capital', 'state'])
d.head(2)

<a id='crosstab'></a>
### Performing Cross-Tabulations

In [ ]:
# Display a cross-tabulation of two Series.
pd.crosstab(users.occupation, users.gender)

<a id='query'></a>
### Query-Filtering Syntax

In [ ]:
# Alternative syntax for Boolean filtering (noted as "experimental" in the documentation):
d = users.query('age < 20')                 # users[users.age < 20]
d = users.query("age < 20 and gender=='M'") # users[(users.age < 20) & (users.gender=='M')]
d = users.query('age < 20 or age > 60')     # users[(users.age < 20) | (users.age > 60)]

<a id='memory-usage'></a>
### Calculating Memory Usage

In [ ]:
# Display the memory usage of a DataFrame.
d = ufo.info()          # Total usage
ufo.memory_usage()  # Usage by column

<a id='category-type'></a>
### Converting Column to Category Type

In [ ]:
# Change a Series to the `category` data type. (This reduces memory usage and increases performance).
ufo['State'] = ufo.State.astype('category')

<a id='assign'></a>
### Creating Columns With `.assign()`

In [ ]:
# Temporarily define a new column as a function of the existing columns.
drinks.assign(servings = drinks.beer + drinks.spirit + drinks.wine).head(2)

<a id='limit-rows-read'></a>
### Limiting the Number of Rows to Load in a File Read

In [ ]:
# Limit which rows are included when reading in a file.
d = pd.read_csv('../datasets/drinks.csv', nrows=10)           # Only read the first 10 rows.
d = pd.read_csv('../datasets/drinks.csv', skiprows=[1, 2])    # Skip the first two rows of data.

<a id='manual-print'></a>
### Manually Setting the Number of Rows and Columns to Print

In [ ]:
# Change the maximum number of rows and columns printed. (`None` means unlimited).
pd.set_option('max_rows', 2)     # Default is 60 rows
pd.set_option('max_columns', 2)  # Default is 20 columns
print drinks

In [ ]:
# Reset the options to defaults.
pd.reset_option('max_rows')
pd.reset_option('max_columns')

In [ ]:
# Change the options temporarily. (Settings are restored when you exit the `with` block).
with pd.option_context('max_rows', None, 'max_columns', None):
    print drinks